This assignment is the first part of two exercises, in which we will analyze LIGO data to find the gravitational wave transient caused by the coalescence of two neutron stars (GW170817).

You are given a true 2048-second segment of Hanford LIGO data, sampled at 4096 Hz (down-sampled from the original 16 kHz data). Along with this PDF, you should have:

1. `strain.npy`, readable by NumPy, containing the strain data.
2. `gw_search_functions`, containing helpful functions, constants.
3. The timestamps corresponding to the strain are not uploaded due to size, and are instead provided in `gw_search_functions`.

---

In this notebook, create an use a **template-bank**, attempt to find the famous GW170817 event, and place confidence in the detection, in the form of a false-alarm rate.

It is advised to get this code from https://github.com/JonathanMushkin/GW_search_tutorial, and use the pyproject.toml to define an environment.

Please contact jonathan.mushkin[at]weizmann.ac.il for any help, question or comment.



In [ ]:
import time
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, stats
import gw_search_functions
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
plt.rcParams["axes.titlesize"] = 16


## 0  
Load data, evaluate ASD and whitening filter

In [ ]:
filename = "strain.npy"
event_name = "GW170817"
detector_name = "H"
fs = 2**12  # Hz

strain = np.load(filename)
times = np.arange(len(strain)) / fs
dt = times[1] - times[0]
freqs = np.fft.rfftfreq(len(strain), d=dt)
df = freqs[1] - freqs[0]

tukey_window = signal.windows.tukey(M=len(strain), alpha=0.1)
strain_f = np.fft.rfft(strain * tukey_window)

seg_duration = 64
overlap_duration = 32
nperseg = int(seg_duration * fs)
noverlap = int(overlap_duration * fs)
welch_dict = {
    "x": strain,
    "fs": fs,
    "nperseg": nperseg,
    "noverlap": noverlap,
    "average": "median",
    "scaling": "density",
}
psd_freqs, psd_estimation = signal.welch(**welch_dict)
asd_estimation = psd_estimation ** (1 / 2)
fmin = 20
asd = np.interp(freqs, psd_freqs, asd_estimation)

# Create high-pass filter
# make it go like sin-squared from 0 to 1 over (fmin, fmin+1Hz) interval
highpass_filter = np.zeros(len(freqs))
i1, i2 = np.searchsorted(freqs, (fmin, fmin + 1))
highpass_filter[i1:i2] = np.sin(np.linspace(0, np.pi / 2, i2 - i1)) ** 2
highpass_filter[i2:] = 1.0

# whitening filter is 1/asd(f) * high-pass filter
whitening_filter_raw = highpass_filter / np.interp(
    x=freqs, xp=psd_freqs, fp=asd_estimation
)

# To avoid ripples in Fourier domain, we apply a windowing in time domain

padded_tukey_window = np.fft.fftshift(
    np.pad(
        signal.windows.tukey(M=nperseg, alpha=0.1),
        pad_width=(len(strain) - nperseg) // 2,
        constant_values=0,
    )
)
# tranform to time domain, apply the window, and return to frequency domain
whitening_filter = (
    highpass_filter
    * np.fft.rfft(padded_tukey_window * np.fft.irfft(whitening_filter_raw))
).real * np.sqrt(2 * dt)

wht_strain_f = strain_f * whitening_filter
wht_strain_t = np.fft.irfft(wht_strain_f)

i1 = np.searchsorted(freqs, fmin)


# 1

 Now you know how to conduct a search with a single template. We now go on to prepare a bank of templates. 
 
 The game is make the template bank "dense" enough so the mismatch between a true signal in the data is never too large, while making it "sparse" enough not to be wasteful. For example, if 2 parameters gives the exact same waveform, it is a shame to include both.

 One option to create a bank is to draw samples of the template parameters, create all templates. Then, evaluate all overlaps between template pairs :

 \begin{equation}
\mathcal{O}_{ij} = \frac{\vert\langle h_i \mid h_j \rangle\vert}{\sqrt{\langle h_i \mid h_i \rangle \langle h_j \mid h_j \rangle}}
 \end{equation}
 time time and phase shifts are done in the search. But the cost of this process is hugh (scales like bank size squared).

If instead we find a good basis to describe the banks, we can evaluate the match / mismatch between templates based on their new found coordinates.

The main idea is that the $h_+$ waveform can be represented by an amplitude and phase. For a small enough region of parameter space, the amplitudes are the same and the phases are a combination of a common phase evolution and a deviation that can be written as a linear combination of orthonormal phase functions:

$$
\Psi_i(f) = \bar{\Psi}(f) + \sum_\alpha c_\alpha \psi_\alpha(f)
$$

We assume the phases are the linear-free (global phase and time standartization)  as discussed in the previous notebook, and is available at
`gw_search_functions.phases_to_linear_free_phases`. 

Here, the common evolution $\bar{\Psi}(f)$ is the mean over samples per frequency. Orthonormality is defined using the whitened-amplitude weights:

$$
\sum_f \frac{A^2(f)}{S_n(f)} \psi_i(f)\psi_j(f) = \delta_{ij}
$$

Given two normalized templates $h_i(f)$, $i=1,2$, the match between the templates is:

$$
\langle h_i| h_j\rangle = \sum_f \frac{A_i(f) A_j(f)}{S_n(f)} e^{i(\Psi_1(f)-\Psi_2(f))} \, \mathrm{d}f
$$

To second order in $\Delta \Psi = \Psi_i - \Psi_j$:

$$
\langle h_i | h_j \rangle \approx \sum_f\frac{A^2(f)}{S_n(f)} \left(1 + i \Delta \Psi(f) - \frac{1}{2}(\Delta\Psi(f))^2 \right) \, \mathrm{d}f
$$

The imaginary part will not matter for the SNR calculation, so we ignore it (return to the Gaussian likelihood to understand why). The second-order term becomes:

$$
\langle h_i | h_j \rangle \approx 1 - \frac{1}{2} \sum_\alpha (\Delta c_\alpha)^2
$$

We can create an orthonormal basis by taking the sample phases and performing SVD on a matrix $X$ of shape $N_{\rm samples} × N_{\rm frequencies}$:

$$
X_{ij} = \Psi_i(f_j) \cdot \frac{A(f_j)}{\sqrt{S_n(f_j)}}
$$

To obtain the desired $\psi_\alpha(f)$, divide the resulting linear basis vectors by the weights. The eigenvalues from the SVD indicate how many components are needed to represent the waveform set accurately.

**Note:** The phase evolution is smooth. You can downsample the frequency grid starting at 20 Hz with steps of $2^{-4}$ Hz to reduce computational cost. Performing SVD on the full-resolution grid may be too demanding for standard hardware.

In [ ]:
m1, m2 = gw_search_functions.draw_mass_samples(2**8)

In [ ]:
# take sparser frequency grid
fslice = slice(np.searchsorted(freqs, (fmin)), len(freqs), 128)
fs = freqs[fslice]
phases = np.array(
    [
        gw_search_functions.masses_to_phases(mm1, mm2, fs)
        for mm1, mm2 in zip(m1, m2)
    ]
)

In [ ]:
wht_amp = (amp * whitening_filter)[fslice]
wht_amp = wht_amp / np.sqrt(np.sum(wht_amp**2))  # renormalize

linear_free_phases = gw_search_functions.phases_to_linear_free_phases(
    phases, freqs[fslice], weights=wht_amp
)
common_phase_evolution = linear_free_phases.mean(axis=0)
fig, ax = plt.subplots()
ax.plot(freqs[fslice], linear_free_phases.T)
_ = ax.plot(freqs[fslice], common_phase_evolution, ls="--", c="k")
ax.text(0.75, 0.95, r"NOT REQUIRED", color="red", transform=ax.transAxes)

In [ ]:
phases_without_common_evolution = linear_free_phases - common_phase_evolution
svd_phase = phases_without_common_evolution
svd_weights = wht_amp
print(svd_weights.shape, svd_phase.shape)


In [ ]:
# could take up to 1-5 minutes.
u, d, v = np.linalg.svd(svd_phase * svd_weights)


In [ ]:
fig, ax = plt.subplots()
ax.semilogy(d[:20], ".")
ax.text(0.5, 0.95, r"NOT REQUIRED", color="red", transform=ax.transAxes)
ax.text(
    0.5,
    0.75,
    r"This is why we take 2 components",
    color="red",
    transform=ax.transAxes,
)
ax.grid()

In [ ]:
# check that eignen vectors has zero weighted mean. Meaning, they are orthogonal to a constant function.
(v[0] * svd_weights).mean(), (v[1] * svd_weights).mean()

In [ ]:
output_dir = Path("local_outputs")
if not output_dir.exists():
    output_dir.mkdir()
np.savez(
    output_dir / "GW170817_H_svd", u=u, d=d, v=v, freqs_sliced=fs, freqs=freqs
)

In [ ]:
u, d, v = [
    np.load(output_dir / "GW170817_H_svd.npz").get(k) for k in ("u", "d", "v")
]

In [ ]:
# pick 2 coordinates
ndim = 2
u = u[:, :ndim]
d = d[:ndim]
v = v[:ndim, :]

# create a phase vector (without weights) from SVD components
# and new set of coordiantes
coordinates = u * d

phase_basis_coarse_freqs = np.zeros_like(v)
mask = svd_weights != 0
phase_basis_coarse_freqs[:, mask] = v[:, mask] / svd_weights[mask]

In [ ]:
# normalization tests
print("NOT REQUIRED")
print("SVD weights norm:", np.sum(svd_weights**2))
print(
    "First basis norm:",
    np.sum(svd_weights**2 * phase_basis_coarse_freqs[0] ** 2),
)
print(
    "Const. component in first basis:",
    np.sum(svd_weights**2 * phase_basis_coarse_freqs[1]),
)
print(
    "Second basis norm:",
    np.sum(svd_weights**2 * phase_basis_coarse_freqs[0] ** 2),
)
print(
    "Const. component in second basis:",
    np.sum(svd_weights**2 * phase_basis_coarse_freqs[0]),
)


In [ ]:
fig, ax = plt.subplots()
ax.plot(phase_basis_coarse_freqs[0], label="1st component")
ax.plot(phase_basis_coarse_freqs[1], label="2nd component")
ax.grid()
leg = ax.legend()


In [ ]:
# create full-frequency resolution phase basis
phase_basis = np.array(
    [
        np.interp(x=freqs, xp=freqs[fslice], fp=phase_base, left=0)
        for phase_base in phase_basis_coarse_freqs
    ]
)

##  8

Calculate the inner product between waveforms at different coordinate-distance $\sqrt{\sum_\alpha |\Delta c_\alpha |^2)}$.
**Plot their overlap against their distance, and the theoretical prediction**.

In [ ]:
distances = 10 ** np.linspace(-2, 0.5, num=30)
amp = np.zeros_like(freqs[fslice])
cond = whitening_filter[fslice] != 0
amp[cond] = freqs[fslice][cond] ** (-7 / 6)
amp_wht = amp * whitening_filter[fslice]
amp_wht /= np.sqrt(np.sum((amp_wht) ** 2))

matches = np.zeros_like(distances)
for i, distance in enumerate(distances):
    dist_per_coordinate = distance / np.sqrt(2)
    coordinate = np.array([dist_per_coordinate, dist_per_coordinate])
    phase = coordinate @ phase_basis_coarse_freqs + common_phase_evolution
    matches[i] = np.sum(
        amp_wht
        * np.exp(-1j * phase)
        * amp_wht
        * np.exp(+1j * common_phase_evolution)
    ).real

In [ ]:
fig, ax = plt.subplots()
ax.loglog(distances**2, 1 - matches, label="$1 - $ match")
ax.loglog(
    distances**2,
    distances**2 / 2,
    ".",
    label=r"$1-\frac{1}{2}\sum_\alpha|\Delta c_\alpha |^2$",
)

ax.set_xlabel(r"distance = $\sum_\alpha |\Delta c_\alpha |^2$")
ax.set_ylabel("mismatch = $1 - $match")
leg = ax.legend(loc="upper left")
ax.grid()

# 9 

Draw $2^{13}$ mass samples. Create the phase for each, and find the coordinates of each.
Select a subset such that the distance between any 2 samples is not smaller than 0.1. 

On the same plot, create a scatter plot of the 213 samples and of the selected subset.
On the plot, write down the size of subset. This subset defines the search bank.

In [ ]:
fslice = slice(np.searchsorted(freqs, (fmin)), len(freqs), 128)
freqs_low_res = freqs[fslice]

m1, m2 = gw_search_functions.draw_mass_samples(2**13)
redshift = 0.01
fig, ax = plt.subplots()
ax.scatter(m1, m2, marker=".", s=1)
ax.scatter(1.46 * (1 + redshift), 1.27 * (1 + redshift), s=100, marker="x")
ax.text(0.5, 0.95, r"NOT REQUIRED", color="red", transform=ax.transAxes)

In [ ]:
phases_on_coarse_freqs = np.array(
    [
        gw_search_functions.masses_to_phases(mm1, mm2, freqs_low_res)
        for mm1, mm2 in zip(m1, m2)
    ]
)
linear_free_phases = gw_search_functions.phases_to_linear_free_phases(
    phases_on_coarse_freqs, freqs_low_res, amp_wht
)

phases_without_common_evolution = linear_free_phases - common_phase_evolution

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1)

axs[0].plot(freqs_low_res, linear_free_phases[:64].T)
axs[0].plot(freqs_low_res, common_phase_evolution, ls="--", c="k")
axs[1].plot(freqs_low_res, phases_without_common_evolution[:64].T)
for ax in axs:
    ax.text(0.5, 0.95, r"NOT REQUIRED", color="red", transform=ax.transAxes)


In [ ]:
coordinates = (
    svd_weights**2 * phases_without_common_evolution
) @ phase_basis_coarse_freqs.T

bank_coordinates, bank_indices = (
    gw_search_functions.select_points_without_clutter(
        coordinates, np.sqrt(0.1)
    )
)

In [ ]:
plt.scatter(
    *coordinates.T,
    s=1,
    alpha=0.5,
    c="r",
    label=f"full set ({len(coordinates)} points)",
)
plt.scatter(
    *bank_coordinates.T,
    s=5,
    c="k",
    label=f"subset ({len(bank_coordinates)} points)",
)
print(bank_coordinates.shape)
plt.legend()

In [ ]:
amp = np.zeros_like(freqs)
amp[i1:] = freqs[i1:] ** (-7 / 6)
normalization = gw_search_functions.correlate(amp, amp, w=whitening_filter)[
    0
] ** (1 / 2)
amp /= normalization

# 10

Repeat the search (sections 4-6, without repeating their plots) for each template in the bank individually (including glitch-removal). For each interval of 0.1 seconds, record which template gave the maximal SNR, and what was that SNR. **Plot the time-series of maximal $\text{SNR}^2$ in per 0.1 seconds. Plot a histogram of the maximal values per 0.1 seconds**. *Before using the entire bank, try a small subset and see that the results make sense. The entire search could take several minutes, depending on hardware*.

In [ ]:
indices_lists = []
snr2_lists = []
min_snr2_to_save = stats.chi2(df=2).isf(1 / (times[-1] / 0.1))
glitch_test_threshold = stats.chi2(df=2).isf(0.01)
snr2_lists_raw = []
indices_lists_raw = []
glitch_mask_list = []
common_phase_evolution_high_res = np.interp(
    x=freqs, xp=freqs_low_res, fp=common_phase_evolution
)
fs = 1 / dt
t_start = time.time()

for template_index, template_coordinate in tqdm(
    enumerate(bank_coordinates), total=len(bank_coordinates), desc="template"
):
    phase = common_phase_evolution_high_res + template_coordinate @ phase_basis
    h = amp * np.exp(1j * phase)

    snr2 = gw_search_functions.snr2_timeseries(
        h * whitening_filter, strain_f * whitening_filter
    )
    h_low, h_high = np.zeros((2, len(freqs)), complex)
    h_low[:i_fbar] = h[:i_fbar]
    h_high[i_fbar:] = h[i_fbar:]
    z_low = gw_search_functions.complex_overlap_timeseries(
        h_low * whitening_filter, strain_f * whitening_filter
    )
    z_high = gw_search_functions.complex_overlap_timeseries(
        h_high * whitening_filter, strain_f * whitening_filter
    )

    glitch_test_statistic = np.abs(z_low - z_high) ** 2

    glitch_mask = (glitch_test_statistic > glitch_test_threshold) * (snr2 > 10)
    glitch_mask_list.append(glitch_mask)
    maxs, argmaxs = gw_search_functions.max_argmax_over_n_samples(
        snr2 * ~glitch_mask, int(0.1 * fs)
    )
    indices_lists.append(argmaxs)
    snr2_lists.append(maxs)

    maxs, argmaxs = gw_search_functions.max_argmax_over_n_samples(
        snr2, int(0.1 * fs)
    )
    indices_lists_raw.append(argmaxs)
    snr2_lists_raw.append(maxs)

snr2_per_template = np.array(snr2_lists)
time_indices_per_template = np.array(indices_lists)
snr2_per_template_with_glitches = np.array(snr2_lists_raw)

print("Done!")

In [ ]:
time_bins = np.linspace(0, times[-1], snr2_per_template.shape[1])


In [ ]:
fig, ax = plt.subplots()
ax.plot(time_bins, snr2_per_template.max(axis=0))
ax.set_xlabel("time (s)")
ax.set_ylabel(r"Bestfit ${\rm SNR}^2$")

In [ ]:
fig, ax = plt.subplots()
hist_kwargs = {"histtype": "step", "density": True, "log": True, "bins": 200}
counts, edges, patches = ax.hist(
    snr2_per_template.max(axis=0),
    **hist_kwargs,
    alpha=0.5,
    label="With glitch removal",
)

hist_kwargs = {"histtype": "step", "density": True, "log": True, "bins": 200}
counts, edges, patches = ax.hist(
    snr2_per_template_with_glitches.max(axis=0).flatten(),
    label="Before glitch removal",
    **hist_kwargs,
    ls="--",
)

ax.set_xlabel(r"${\rm SNR}^2$")
ax.set_ylabel("counts (normalized)")
leg = ax.legend()


## 11.
If you detected an event, **report its time, the masses of the template and an estimation or a upper bound of the false-alarm rate for such SNR**. Consider the number of templates you used and the fact that waveforms have typical auto-correlation length of 1 ms.

In [ ]:
# compare with https://arxiv.org/pdf/1710.05832 Table 1
best_template_index, best_timestamp_index = np.unravel_index(
    snr2_per_template.argmax(), snr2_per_template.shape
)
bestfit_m1 = m1[best_template_index]
bestfit_m2 = m2[best_template_index]
bestfit_mchirp = gw_search_functions.m1m2_to_mchirp(bestfit_m1, bestfit_m2)
bestfit_snr2 = snr2_per_template.max()
time_bins = np.linspace(0, times[-1], snr2_per_template.shape[1])
bestfit_time = snr2_per_template.max(axis=0).argmax()

print("NOT REQUIRED")
print(f"Maximal SNR^2 found : {bestfit_snr2:.5g} at time {bestfit_time:.3g}")
print(
    f"Template of masses ({bestfit_m1:.3g},{bestfit_m2:.3g}), or chirp-mass {bestfit_mchirp:.5g} (solar masses)"
)


p = Probability that non of N = N_templates * N_times individual experiments will reach value x or higher :
$$p = 1 - (SF(x))^N$$

Probability that at least one of $N$ experiments will reach value x:
$1 - p = (SF(x))^N $

At this high SNR^2, the FAR is almost exactly zero


In [ ]:
N_templates = bank_coordinates.shape[0]
N_trials = N_templates * len(times)

stats.chi2(df=2).sf(snr2_per_template.max()) ** N_trials

In [ ]:
np.log(1 - stats.chi2(df).sf(snr2_per_template.max()))

use binomial to imporve the calculation

$$ FAR = 1 - CDF(x)^N = 1 - (1-SF(x))^N \approx 1 - 1 + N\cdot SF(x) = N\cdot SF(x) $$

In [ ]:
snr2_per_template.size * stats.chi2(df=2).sf(snr2_per_template.max())

## 12.

**Create a spectogram (using e.g. `matplotlib.pyplot.specgram`), localized in time and frequency around the event you found**.

In [ ]:
# create the histogram in 2 steps. So I can calibrate the dynamic range in the second histogram using the fist histogram
specgram_kwargs = {
    "x": np.fft.irfft(strain_f * whitening_filter),
    "NFFT": int(fs * 0.5),
    "noverlap": int(fs * 0.25),
    "scale": "linear",
    "vmin": 0,
    "vmax": 25,
    "Fs": fs,
}

o = plt.specgram(**specgram_kwargs)

In [ ]:
specgram_kwargs = {
    "x": np.fft.irfft(strain_f * whitening_filter),
    "NFFT": int(fs * 0.5),
    "noverlap": int(fs * 0.25),
    "scale": "linear",
    "vmin": 0,
    "vmax": o[0][(o[1] > 20) * (o[1] < 1000)].std() * 5,
    "Fs": fs,
}

o = plt.specgram(**specgram_kwargs)
tmin = 0.1 * 10259 - 6
tmax = 0.1 * 10259 + 1
fmin = 20
fmax = 1000
plt.xlim(tmin, tmax)
plt.ylim(20, 1000)

In [ ]:
TIME1 = time.time()

print(f"Time passed: {TIME1 - TIME0:.3g} seconds")